In [1]:
from bs4 import BeautifulSoup
import requests
from splinter import Browser

In [2]:
# Showing the chromdriver I'm using
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
# Executable path and fire up browser, headless set to True so the browser operates in the background invisible 
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [5]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [6]:
# Find title and place it into a variable
news_title = soup.find('div', class_="content_title").text
print(news_title)

A Rover Pit Stop at JPL


In [7]:
# Find paragraph text and put it into a variable
news_p = soup.find('div', class_="article_teaser_body").text
print(news_p)

Working like a finely honed machine, a team of engineers in this time-lapse video clip install test wheels on another finely honed machine: NASA's Mars 2020 rover.


In [8]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [9]:
browser.click_link_by_partial_text('FULL IMAGE')

In [10]:
browser.click_link_by_partial_text('more info')

In [11]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [22]:
image = soup.find('img', class_= "main_image")
featured_image_url = "https://www.jpl.nasa.gov" + image["src"]

In [29]:
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)

In [30]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [32]:
mars_weather = soup.find('span', class_="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0").text
print(mars_weather)

AttributeError: 'NoneType' object has no attribute 'text'